In [ ]:
# 找基准线
import cv2
import numpy as np
import open3d as o3d
# 点云列表
point_data =[]
de =[]
dem =[]
n =100  #751
for j in range(455,459):
    i_  = j*1000
    imageN = cv2.imread('./cap_v2/{}_diffN.bmp'.format(i_))
    imageP = cv2.imread('./cap_v2/{}_diffP.bmp'.format(i_)) 
    print(j)
    diff = cv2.subtract(imageP, imageN)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
    m =[]
    for i in range(gray.shape[0]): #行数
        # 找到每一行最大亮度的索引
        #print(np.argmax(gray[i]),np.max(gray[i]))
        if np.max(gray[i])>20:  #10可以 5可以
            # 将最大值像素 置为255
            gray[i][np.argmax(gray[i])]=255
            # 其余的像素置为0
            gray[i][:np.argmax(gray[i])]=0
            gray[i][np.argmax(gray[i])+1:]=0
            m.append(np.argmax(gray[i]))
        else:
            # 其余的像素置为0
            gray[i][:]=0
    baseline = j
    d =[]
    print(min(m)) #这里为1096
    for i in range(gray.shape[0]): #行数
        if np.max(gray[i]) != 0:
            deep =np.argmax(gray[i])- 1096#1096
            #deep = min(max(np.argmax(gray[i])-1096,0),60) # 保持在-100到100之间
            if -4 <=(deep)<=70: #高度
                point_data.append([i,np.argmax(gray[i])-baseline,deep ])
                d.append(deep)
        gray[i][min(m)]=255 #显示用

    #print(d)
#     de.append(min(d))
#     dem.append(max(d))
# print(de)
# print(dem)
# print(min(de),max(dem))


print(point_data)
# 将点数据转换为NumPy数组
points = np.array(point_data)
# 创建一个点云对象
pcd = o3d.geometry.PointCloud()
# 将NumPy数组中的点数据设置为点云的点
pcd.points = o3d.utility.Vector3dVector(points)
# 可视化点云
#显示坐标轴
#coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0]) #x红 y绿 z蓝
o3d.visualization.draw_geometries([pcd])
# 如果需要，可以将点云保存为PCD文件
o3d.io.write_point_cloud("output_{}.pcd".format(n), pcd)

In [12]:

# 中心法提取中心线
import cv2
import numpy as np

# 读取图像
src_img = cv2.imread("diff_image.jpg", 0)

# 克隆原图像并转换为RGB，以便在其上绘制彩色点
dst_img = cv2.cvtColor(src_img, cv2.COLOR_GRAY2RGB)

# 获取图像的尺寸
rows, cols = src_img.shape[:2]

pts = []

for i in range(cols):
    # 初始化最小和最大y值
    y_min, y_max = None, None

    for j in range(rows):
        # 寻找每列的第一个非零像素
        if src_img[j, i] != 0:
            y_min = j
            break

    for j in range(rows - 1, -1, -1):
        # 寻找每列的最后一个非零像素
        if src_img[j, i] != 0:
            y_max = j
            break

    # 如果找到有效的y_min和y_max，则计算中心点并存储
    if y_min is not None and y_max is not None and y_min < y_max:
        pts.append((i, (y_min + y_max) // 2))

# 在每个中心点位置绘制红色圆点
for pt in pts:
    cv2.circle(dst_img, pt, 1, (0, 0, 255), -1)

# 保存修改后的图像
cv2.imwrite("diff_image_1.jpg", dst_img)


True

In [15]:
# 中心法提取中心线
import cv2
import numpy as np

# 读取图像
diff = cv2.imread("diff_image.jpg", 0)
# 找出轮廓
contours, _ = cv2.findContours(diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 绘制轮廓
cv2.drawContours(diff, contours, -1, (255, 255, 255), 1)



# 保存修改后的图像
cv2.imwrite("diff_image_1.jpg", diff)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\contours.cpp:194: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [20]:
import cv2
import numpy as np

# 读取图像
src_img = cv2.imread("diff_image.jpg", 0)

# 确保图像为灰度图
if len(src_img.shape) > 2:
    src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)

# 图像预处理，例如去噪
#src_img = cv2.GaussianBlur(src_img, (5, 5), 0)

# 边缘检测
edges = cv2.Canny(src_img, 50, 150, apertureSize=3)

# 寻找轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 遍历每个轮廓
for contour in contours:
    # 计算轮廓的矩
    moments = cv2.moments(contour) #
    
    # 计算中心点（重心）
    if moments['m00'] != 0:
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
        
        # 在原图上绘制中心点
        cv2.circle(src_img, (cx, cy), 5, (255, 0, 0), -1)
        print(cx,cy)

# 显示结果
# cv2.imshow('Center of Gravity', src_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# 保存修改后的图像
cv2.imwrite("diff_image_1.jpg", src_img)


1063 1920
1064 1891
1055 1874
1048 1262
1045 1019
1047 879
1050 605
1023 160
1063 7


True

In [31]:
from skimage import morphology
import cv2
import numpy as np

def skeleton_demo(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) #二值化
    cv2.imwrite("diff_image_binary.jpg", binary)
    binary[binary == 255] = 1 #
    skeleton0 = morphology.skeletonize(binary) #骨架化
    skeleton = skeleton0.astype(np.uint8) * 255
    contours, hierarchy = cv2.findContours(skeleton, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 1, 8)
    cv2.imwrite("diff_image_1.jpg", image)
    # cv2.imshow("skeleton", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

def medial_axis_demo(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    binary[binary == 255] = 1
    skel, distance = morphology.medial_axis(binary, return_distance=True)
    dist_on_skel = distance * skel
    skel_img = dist_on_skel.astype(np.uint8) * 255
    contours, hierarchy = cv2.findContours(skel_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 1, 8) #-1
    cv2.imwrite("diff_image_2.jpg", image)
    # cv2.imshow("result", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# # 调用示例
image = cv2.imread("diff_image.jpg")
medial_axis_demo(image)

# 调用示例
image = cv2.imread("diff_image.jpg")
skeleton_demo(image)


In [ ]:
import cv2
diff = cv2.imread("diff_image.jpg")
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
# 应用高斯模糊减少噪声和细节
gray = cv2.GaussianBlur(gray, (5, 5), 0) #（5,5）是高斯核的大小，0是标准差


In [82]:
# 中心法提取中心线  !!!
import cv2
import numpy as np

import cv2
diff = cv2.imread("diff_image.jpg")
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
# 应用高斯模糊减少噪声和细节
gray = cv2.GaussianBlur(gray, (7, 7), 0) #（5,5）是高斯核的大小，0是标准差



for i in range(gray.shape[0]):
    # 初始化最小和最大y值
    y_min, y_max = None, None

    for j in range(gray.shape[1]):
        # 寻找每列的第一个非零像素
        if gray[i, j] >= 20:
            y_min = j
            break

    for j in range(gray.shape[1] - 1, -1, -1):
        # 寻找每列的最后一个非零像素
        if src_img[i, j] >= 20:
            y_max = j
            break

    # 如果找到有效的y_min和y_max，则计算中心点并存储
    if y_min is not None and y_max is not None and y_min < y_max:
        #pts.append((i, (y_min + y_max) // 2))
        diff[i][(y_min + y_max) // 2]=[255, 255, 255] #白色



# 保存修改后的图像
cv2.imwrite("diff_image_1.jpg", diff)

KeyboardInterrupt: 

In [2]:
# 极值法提取中心线！！！
import cv2
import numpy as np
diff = cv2.imread("diff_image.jpg")
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
# 应用高斯模糊减少噪声和细节
gray = cv2.GaussianBlur(gray, (5, 5), 0) #（5,5）是高斯核的大小，0是标准差

# Canny边缘检测
#gray = cv2.Canny(gray, 0, 30, apertureSize=3) #50表示低阈值，150表示高阈值 低于50的像素点会被抛弃，高于150的像素点会被保留
m =[]
for i in range(gray.shape[0]): #行数
    # 找到每一行最大亮度的索引
    #print(np.argmax(gray[i]),np.max(gray[i]))
    if np.max(gray[i])>5:  #10可以 5可以
        # 将最大值像素 置为255
        gray[i][np.argmax(gray[i])]=255
        # 在原图上显示
        diff[i][np.argmax(gray[i])]=[255, 255, 255] #白色

    #
        # 其余的像素置为0
        gray[i][:np.argmax(gray[i])]=0
        gray[i][np.argmax(gray[i])+1:]=0
        m.append(np.argmax(gray[i]))
    else:
        # 其余的像素置为0
        gray[i][:]=0

    
cv2.imwrite("diff_image_binary_.jpg", gray)
cv2.imwrite("diff_image_4.jpg", diff)




True

In [17]:
# 灰度重心法 提取中心线
import cv2
import numpy as np
i_  = 457*1000
imageN = cv2.imread('./cap_v2/{}_diffN.bmp'.format(i_))
imageP = cv2.imread('./cap_v2/{}_diffP.bmp'.format(i_)) 
diff = cv2.subtract(imageP, imageN)
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
# 应用高斯模糊减少噪声和细节
gray = cv2.GaussianBlur(gray, (5, 5), 0) #（5,5）是高斯核的大小，0是标准差

# Canny边缘检测
#gray = cv2.Canny(gray, 0, 30, apertureSize=3) #50表示低阈值，150表示高阈值 低于50的像素点会被抛弃，高于150的像素点会被保留
m =[]
for i in range(gray.shape[0]): #行数
    # 找到每一行最大亮度的索引
    #print(np.argmax(gray[i]),np.max(gray[i]))
    fenzi = fenmu = 0
    if np.max(gray[i])>5:  #10可以 5可以
        max_ind = np.argmax(gray[i])
        for j in range(max_ind-10,max_ind+10):
            fenzi = fenzi + j*gray[i][j]
            fenmu = fenmu + gray[i][j]
        center_ind = fenzi/fenmu #重心


        # #四舍五入
        center_ind = round(center_ind)
        # 将最大值像素 置为255
        gray[i][center_ind]=255
        # 在原图上显示
        diff[i][center_ind]=[255, 255, 255] #白色

    #
        # 其余的像素置为0
        gray[i][:center_ind]=0
        gray[i][center_ind+1:]=0
        m.append(center_ind)
    else:
        # 其余的像素置为0
        gray[i][:]=0

    
#cv2.imwrite("diff_image_binary_.jpg", gray)
cv2.imwrite("diff_457.bmp", diff)


True

In [1]:
# 第3版 灰度重心法
import cv2
import numpy as np
import open3d as o3d
# 点云列表
point_data =[]
de =[]
dem =[]
n =751
for b in range(751):
    i_  = b*1000
    imageN = cv2.imread('./cap_v2/{}_diffN.bmp'.format(i_))
    imageP = cv2.imread('./cap_v2/{}_diffP.bmp'.format(i_)) 
    print(b)
    diff = cv2.subtract(imageP, imageN)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
    gray = cv2.GaussianBlur(gray, (5, 5), 0) #（5,5）是高斯核的大小，0是标准差
    m =[]
    for i in range(gray.shape[0]): #行数
        # 找到每一行最大亮度的索引
        #print(np.argmax(gray[i]),np.max(gray[i]))
        fenzi = fenmu = 0
        if np.max(gray[i])>5:  #10可以 5可以
            max_ind = np.argmax(gray[i])
            for j in range(max_ind-10,max_ind+10):
                fenzi = fenzi + j*gray[i][j]
                fenmu = fenmu + gray[i][j]
            center_ind = fenzi/fenmu #重心

            m.append(center_ind)
            # #四舍五入
            center_ind = round(center_ind)
            # 将最大值像素 置为255
            gray[i][center_ind]=255
            # 在原图上显示
            diff[i][center_ind]=[255, 255, 255] #白色

        #
            # 其余的像素置为0
            gray[i][:center_ind]=0
            gray[i][center_ind+1:]=0
            
        else:
            # 其余的像素置为0
            gray[i][:]=0
            m.append(np.inf)
    #cv2.imwrite("diff_{}.jpg".format(j), diff)
    baseline = b
    d =[]
    print(min(m)) 
    #print(len(m))
    #print(min(m)) #这里为1096
    for i in range(gray.shape[0]): #行数
        if np.max(gray[i]) != 0:
            deep =m[i] - (min(m))#1312   #1096#1096
            #print(m[i])
            #deep = min(max(np.argmax(gray[i])-1096,0),60) # 保持在-100到100之间
            if -4 <=(deep)<=70: #高度
                point_data.append([i,np.argmax(gray[i])-baseline,deep ])  #np.argmax(gray[i])
                d.append(deep)
        #gray[i][min(m)]=255 #显示用
    #print(d)
#     de.append(min(d))
#     dem.append(max(d))
# print(de)
# print(dem)
# print(min(de),max(dem))


#print(point_data)
# 将点数据转换为NumPy数组
points = np.array(point_data)
# 创建一个点云对象
pcd = o3d.geometry.PointCloud()
# 将NumPy数组中的点数据设置为点云的点
pcd.points = o3d.utility.Vector3dVector(points)
# 可视化点云
#显示坐标轴
#coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0]) #x红 y绿 z蓝
o3d.visualization.draw_geometries([pcd])
# 如果需要，可以将点云保存为PCD文件
o3d.io.write_point_cloud("output_v2_{}_z.pcd".format(n), pcd)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0
1312.438596491228
1
1312.2631578947369
2
1312.6451612903227
3
1312.3728813559321
4
1312.3684210526317
5
1312.4259259259259
6
1312.402985074627
7
1311.4117647058824
8
1311.7
9
1311.590909090909
10
1311.25
11
1310.952380952381
12
1311.2631578947369
13
1311.765625
14
1311.8253968253969
15
1311.5
16
1311.45
17
1311.679487179487
18
1311.9454545454546
19
1312.1333333333334
20
1312.0833333333333
21
1311.875
22
1311.52
23
1311.8714285714286
24
1311.9166666666667
25
1311.5068493150684
26
1311.7796610169491
27
1311.876712328767
28
1311.6976744186047
29
1312.047619047619
30
1312.0142857142857
31
1311.4
32
1311.5714285714287
33
1310.9310344827586
34
1311.725806451613
35
1312.0
36
1312.3392857142858
37
1311.75
38
1312.1666666666667
39
1312.076923076923
40
1311.7241379310344
41
1311.7333333333333
42
1312.0277777777778
43
1311.67948

True

In [38]:
# 第2版 极值法
import cv2
import numpy as np
import open3d as o3d
# 点云列表
point_data =[]
de =[]
dem =[]
n ="50-600"
for j in range(50,600):
    i_  = j*1000
    imageN = cv2.imread('./cap_v2/{}_diffN.bmp'.format(i_))
    imageP = cv2.imread('./cap_v2/{}_diffP.bmp'.format(i_)) 
    print(j)
    diff = cv2.subtract(imageP, imageN)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # 转换为灰度图像  #0是黑色，255是白色
    gray = cv2.GaussianBlur(gray, (5, 5), 0) #（5,5）是高斯核的大小，0是标准差
    m =[]
    for i in range(gray.shape[0]): #行数
        # 找到每一行最大亮度的索引
        #print(np.argmax(gray[i]),np.max(gray[i]))
        if np.max(gray[i])>5:  #10可以 5可以
            # 将最大值像素 置为255
            gray[i][np.argmax(gray[i])]=255
            # 在原图上显示
            diff[i][np.argmax(gray[i])]=[255, 255, 255] #白色
            # 其余的像素置为0
            gray[i][:np.argmax(gray[i])]=0
            gray[i][np.argmax(gray[i])+1:]=0
            m.append(np.argmax(gray[i]))
        else:
            # 其余的像素置为0
            gray[i][:]=0
    #cv2.imwrite("diff_{}.jpg".format(j), diff)
    baseline = j
    d =[]
    print(min(m)) 
    #print(min(m)) #这里为1096
    for i in range(gray.shape[0]): #行数
        if np.max(gray[i]) != 0:
            deep =np.argmax(gray[i])-   min(m)#1312   #1096#1096
            #deep = min(max(np.argmax(gray[i])-1096,0),60) # 保持在-100到100之间
            if -4 <=(deep)<=70: #高度
                point_data.append([i,np.argmax(gray[i])-baseline,deep ]) 
                d.append(deep)
        gray[i][min(m)]=255 #显示用
    #print(d)
#     de.append(min(d))
#     dem.append(max(d))
# print(de)
# print(dem)
# print(min(de),max(dem))


#print(point_data)
# 将点数据转换为NumPy数组
points = np.array(point_data)
# 创建一个点云对象
pcd = o3d.geometry.PointCloud()
# 将NumPy数组中的点数据设置为点云的点
pcd.points = o3d.utility.Vector3dVector(points)
# 可视化点云
#显示坐标轴
#coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0]) #x红 y绿 z蓝
o3d.visualization.draw_geometries([pcd])
# 如果需要，可以将点云保存为PCD文件
o3d.io.write_point_cloud("output_v2_{}_j.pcd".format(n), pcd)

50
1312
51
1312
52
1312
53
1312
54
1312
55
1312
56
1312
57
1312
58
1312
59
1312
60
1312
61
1312
62
1312
63
1312
64
1312
65
1312
66
1312
67
1312
68
1312
69
1312
70
1312
71
1312
72
1312
73
1312
74
1312
75
1312
76
1313
77
1313
78
1312
79
1312
80
1312
81
1312
82
1312
83
1312
84
1312
85
1312
86
1312
87
1312
88
1312
89
1312
90
1312
91
1312
92
1312
93
1312
94
1312
95
1312
96
1312
97
1312
98
1312
99
1312
100
1312
101
1312
102
1312
103
1312
104
1312
105
1312
106
1312
107
1312
108
1312
109
1312
110
1312
111
1312
112
1312
113
1313
114
1313
115
1313
116
1313
117
1313
118
1313
119
1312
120
1312
121
1312
122
1313
123
1312
124
1312
125
1310
126
1312
127
1312
128
1312
129
1312
130
1312
131
1312
132
1312
133
1312
134
1312
135
1312
136
1312
137
1312
138
1312
139
1312
140
1312
141
1312
142
1312
143
1312
144
1312
145
1312
146
1312
147
1312
148
1312
149
1312
150
1312
151
1312
152
1312
153
1312
154
1312
155
1312
156
1312
157
1312
158
1312
159
1312
160
1312
161
1312
162
1312
163
1312
164
1312
165
1312
166
13

True

In [66]:
import cv2
import numpy as np

def find_center_line(image_path):
    # 读取图像
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read image.")
        return

    # 转换为灰度图像
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 应用高斯模糊减少噪声和细节
    gray = cv2.GaussianBlur(gray, (3, 3), 0) #（5,5）是高斯核的大小，0是标准差

    # Canny边缘检测
    edges = cv2.Canny(gray, 0, 30, apertureSize=3) #50表示低阈值，150表示高阈值 低于50的像素点会被抛弃，高于150的像素点会被保留
    

    # 使用霍夫线变换检测线条  #1表示距离精度，np.pi / 180表示角度精度，100表示阈值，minLineLength=10表示线段的最小长度，maxLineGap=10表示线段之间的最大间隔
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 35, minLineLength=5, maxLineGap=5) #最后两个参数分别表示线段的最小长度和最大间隔
# 100表示 阈值，低于100的线段会被抛弃，高于100的线段会被保留 单位是像素
    # 创建用于绘制线条的结果图像
    result_image = image.copy()

    # 遍历每条线，绘制线条
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 显示原始图像和结果图像
    #保存
    cv2.imwrite("diff_image_3.jpg", result_image)
    # cv2.imshow('Original Image', image)
    # cv2.imshow('Detected Lines', result_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# 使用函数
find_center_line('diff_image.jpg')


[0 0 0 ... 0 0 0]


In [4]:
import open3d as o3d
# 裁剪点云
pcd = o3d.io.read_point_cloud("output_v2_751_.pcd")
o3d.visualization.draw_geometries_with_editing([pcd])
 

# print("手动裁剪点云示例")

# print("按键 K 锁住点云，并进入裁剪模式")
# print("用鼠标左键拉一个矩形框选取点云，或者用 《ctrl+左键单击》 连线形成一个多边形区域")
# print("按键 C 结束裁剪并保存点云")
# print("按键 F 解除锁定，恢复自由查看点云模式")
# print("17873123809")

 
 

 

In [6]:
import open3d as o3d
pcd = o3d.io.read_point_cloud("cropped_z.ply")
o3d.visualization.draw_geometries_with_editing([pcd])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


KeyboardInterrupt: 